In [1]:
from pyspark.rdd import RDD
from pyspark.sql import Row
from pyspark.sql import DataFrame
from pyspark.sql import SparkSession
import numpy as np
import math
import pandas as pd

import random
import logging
logging.basicConfig(level=logging.INFO)


def init_spark():
    spark = SparkSession \
            .builder \
            .config("spark.driver.memory", "50g") \
            .appName("Python Spark SQL basic example") \
            .config("spark.some.config.option", "some-value") \
            .getOrCreate()
    return spark

spark = init_spark()

df2 = pd.read_csv("C:/Users/vansh/Desktop/Poject big data/anime.csv")
df = spark.read.text("C:/Users/vansh/Desktop/Poject big data/sample1.csv").rdd
data = df.map(lambda x: x[0].split(','))
header1=data.first()
data=data.filter(lambda x:x !=header1)
data2 = data.groupBy(lambda x : x[1])
data3 = data2.collect()



In [2]:
data3

[('20', <pyspark.resultiterable.ResultIterable at 0x230e34a4d88>),
 ('24', <pyspark.resultiterable.ResultIterable at 0x230e34a4d48>),
 ('79', <pyspark.resultiterable.ResultIterable at 0x230e34b25c8>),
 ('226', <pyspark.resultiterable.ResultIterable at 0x230e34b1e08>),
 ('241', <pyspark.resultiterable.ResultIterable at 0x230e34c2048>),
 ('355', <pyspark.resultiterable.ResultIterable at 0x230e34c3248>),
 ('356', <pyspark.resultiterable.ResultIterable at 0x230e34c49c8>),
 ('442', <pyspark.resultiterable.ResultIterable at 0x230e34a4dc8>),
 ('487', <pyspark.resultiterable.ResultIterable at 0x230e34cdb88>),
 ('846', <pyspark.resultiterable.ResultIterable at 0x230e34cba48>),
 ('936', <pyspark.resultiterable.ResultIterable at 0x230e34cfb88>),
 ('1546', <pyspark.resultiterable.ResultIterable at 0x230e34d1048>),
 ('1692', <pyspark.resultiterable.ResultIterable at 0x230e34d1548>),
 ('1836', <pyspark.resultiterable.ResultIterable at 0x230e34d1688>),
 ('2001', <pyspark.resultiterable.ResultIterable